# P9 Produire une étude de marché avec Python
## Partie 1 : Chargement et préparation des données

### importation des packages et chargement des données

In [1]:
import functions
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import numpy as np
import os
import pandas as pd
import seaborn as sns


from matplotlib.gridspec import GridSpec

from pylab import rcParams

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.stats import skew
from scipy.spatial.distance import cdist

from sklearn import cluster
from sklearn import decomposition
from sklearn import preprocessing

from mpl_toolkits.mplot3d import Axes3D



In [2]:
# un dataframe avec la population des pays au cours du temps
df_pop = pd.read_csv('Population_2000_2018.csv')

# un dataframe avec la disponibilité alimentaire et d'autres indicateurs
df_dispo = pd.read_csv('DisponibiliteAlimentaire_2017.csv')

# un dataframe avec les pays et leurs Pib
df_pib = pd.read_excel('pib.xlsx')

# un dataframe avec le coût de la production
df_coût_prod = pd.read_excel('prix_prod.xlsx')

# un dataframe avec les pays et leur continent
df_region = pd.read_excel('continent.xlsx')

## création d'un dataframe exploitable

In [3]:
# isolement des données sur la volaille
df_dispo = df_dispo[df_dispo['Produit']=='Viande de Volailles']

In [4]:
# liste des pays
df_pays_region = df_pop['Zone']
df_pays_region = df_pays_region.drop_duplicates()
df_pays_region = df_pays_region.reset_index()

In [5]:
# l'augmentation du nombre d'habitants par an
df_pays_region = df_pays_region.assign(Groth=0)
for i,j in zip(df_pays_region['Zone'],range(len(df_pays_region))):   
    m, b = np.polyfit(df_pop[df_pop['Zone']==i]['Année'],df_pop[df_pop['Zone']==i]['Valeur'],1)
    df_pays_region['Groth'][j] = m
df_pays_region = df_pays_region[['Zone','Groth']]

<ipython-input-5-93ba4d893420>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pays_region['Groth'][j] = m


In [6]:
# remplacer le nom "groth"
df_pays_region = df_pays_region.rename(columns={'Groth':"Croissance (en millier d'individus)"})

In [7]:
# la production
df_pays_region = df_pays_region.merge(df_dispo[df_dispo['Élément']=='Production'][
    ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':'Production(en millier de tonnes)'})

In [8]:
# la disponibilité intérieure
df_pays_region = df_pays_region.merge(df_dispo[df_dispo['Élément']=='Disponibilité intérieure'][
    ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':'Dispo intérieure(en millier de tonnes)'})

In [9]:
# la disponibilité alimentaire
df_pays_region = df_pays_region.merge(df_dispo[df_dispo[
    'Élément']=='Disponibilité alimentaire en quantité (kg/personne/an)'][
        ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':'Disponibilité alimentaire(en Kg/personne/an)'})

In [10]:
# les exportations
df_pays_region = df_pays_region.merge(df_dispo[df_dispo['Élément']=='Exportations - Quantité'][
    ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':'Exportations (en milier de tonnes)'})

In [11]:
# les importations
df_pays_region = df_pays_region.merge(df_dispo[df_dispo['Élément']=='Importations - Quantité'][
    ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':'Importations (en milier de tonnes)'})

In [12]:
# la population
df_pays_region = df_pays_region.merge(df_pop[df_pop['Année']==2017][
    ['Zone','Valeur']],how='left',left_on=['Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':"Population en 2017 (en milier d'individus)"})

In [13]:
# le pib par habitant
df_pays_region = df_pays_region.merge(df_pib[['Zone','Valeur']],how='left',left_on=[
    'Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Valeur':"Pib par habitant"})
df_pays_region['Pib par habitant']=df_pays_region['Pib par habitant']/df_pays_region[
    "Population en 2017 (en milier d'individus)"]

In [14]:
# la region
df_pays_region = df_pays_region.merge(df_region[['Zone','Continent']],how='left',left_on=[
    'Zone'],right_on=['Zone'])
df_pays_region = df_pays_region.rename(columns={'Continent':"Région"})

In [15]:
# convertir le pays en index
df_pays_region.set_index('Zone',inplace = True)

### traitement des valeurs manquantes

In [16]:
# combien de valeurs manquantes sur chaque colonne ?
df_pays_region.isna().sum()

Croissance (en millier d'individus)               0
Production(en millier de tonnes)                 70
Dispo intérieure(en millier de tonnes)           68
Disponibilité alimentaire(en Kg/personne/an)     66
Exportations (en milier de tonnes)              103
Importations (en milier de tonnes)               68
Population en 2017 (en milier d'individus)        2
Pib par habitant                                 29
Région                                            1
dtype: int64

In [17]:
# suppression de Zone qui ne sont pas considérés comme des pays
df_pays_region = df_pays_region[df_pays_region['Pib par habitant'].isnull()==False]
df_pays_region = df_pays_region[df_pays_region['Dispo intérieure(en millier de tonnes)'].isnull()==False]

In [18]:
# valeurs manquantes de production 
df_pays_region[df_pays_region['Production(en millier de tonnes)'].isnull()]
# Djibouti et les Maldives sont de petits pays, ces pays n'ont pas de production de poulet

,Croissance (en millier d'individus),Production(en millier de tonnes),Dispo intérieure(en millier de tonnes),Disponibilité alimentaire(en Kg/personne/an),Exportations (en milier de tonnes),Importations (en milier de tonnes),Population en 2017 (en milier d'individus),Pib par habitant,Région
Zone,,,,,,,,,
Djibouti,13,NaN,3.0,2.68,NaN,3.0,944.099,2.930697,Afrique
Maldives,12,NaN,12.0,13.50,NaN,12.0,496.402,9.577354,Asie


In [ ]:
df_pays_region[df_pays_region['Exportations (en milier de tonnes)'].isnull()][
    'Production(en millier de tonnes)']+df_pays_region[df_pays_region[
        'Exportations (en milier de tonnes)'].isnull()][
            'Importations (en milier de tonnes)']-df_pays_region[df_pays_region[
                'Exportations (en milier de tonnes)'].isnull()][
                    'Dispo intérieure(en millier de tonnes)']

In [19]:
# valeurs manquantes en exportation
# je fais le cacul suivant : production + importations - disponibilité
# je m'attend à trouver soit 0, soit une petite valeur (lié aux pertes / variations de stock), 
# soit une grande valeur : les exportations
df_pays_region[df_pays_region['Exportations (en milier de tonnes)'].isnull()][
    'Production(en millier de tonnes)']+df_pays_region[df_pays_region[
        'Exportations (en milier de tonnes)'].isnull()][
            'Importations (en milier de tonnes)']-df_pays_region[df_pays_region[
                'Exportations (en milier de tonnes)'].isnull()][
                    'Dispo intérieure(en millier de tonnes)']

Zone
Afghanistan                         0.0
Bahamas                             4.0
Bangladesh                         -1.0
Burkina Faso                        0.0
Cabo Verde                          3.0
Cambodge                            0.0
Cuba                               -1.0
Djibouti                            NaN
Éthiopie                            1.0
Gambie                             10.0
Grenade                             0.0
Guinée                              0.0
Guinée-Bissau                       3.0
Haïti                               0.0
Kiribati                            0.0
Madagascar                         -1.0
Malawi                             32.0
Maldives                            NaN
Mali                               -3.0
Mauritanie                          7.0
Mongolie                            1.0
Mozambique                          0.0
Népal                               0.0
Nouvelle-Calédonie                 -1.0
République centrafricaine          

In [20]:
# combien de valeurs manquantes pour chaque colonne
df_pays_region.isna().sum()

Croissance (en millier d'individus)              0
Production(en millier de tonnes)                 2
Dispo intérieure(en millier de tonnes)           0
Disponibilité alimentaire(en Kg/personne/an)     0
Exportations (en milier de tonnes)              35
Importations (en milier de tonnes)               0
Population en 2017 (en milier d'individus)       0
Pib par habitant                                 0
Région                                           0
dtype: int64

In [21]:
df_pays_region = df_pays_region.fillna(0.0)

### rajouter des données aux dataframe

In [22]:
# remplacer import et export par la balance commerciale
df_pays_region['Exportations (en milier de tonnes)'] = df_pays_region[
    'Exportations (en milier de tonnes)']-df_pays_region['Importations (en milier de tonnes)']
df_pays_region = df_pays_region[["Croissance (en millier d'individus)",'Production(en millier de tonnes)','Dispo intérieure(en millier de tonnes)','Disponibilité alimentaire(en Kg/personne/an)','Importations (en milier de tonnes)',"Population en 2017 (en milier d'individus)",'Pib par habitant','Région']]
df_pays_region = df_pays_region.rename(columns={'Importations (en milier de tonnes)':'Balance commeciale (en milier de tonnes)'})

In [23]:
# le % d'accroissement de la population
df_pays_region["% de croissance"] = df_pays_region["Croissance (en millier d'individus)"]/df_pays_region["Population en 2017 (en milier d'individus)"]*100

In [24]:
# la production par habitant
df_pays_region["Production(en tonne par milier d'habitant)"] = df_pays_region['Production(en millier de tonnes)']/df_pays_region[
    "Population en 2017 (en milier d'individus)"]*1000

In [25]:
# la balance commerciale par habitant
df_pays_region["Balance commerciales(en tonne par milier d'habitant)"] = df_pays_region[
    'Balance commeciale (en milier de tonnes)']/df_pays_region["Population en 2017 (en milier d'individus)"]*1000

### Transformer le continent en valeur numérique grâce à la méthode du one hot encoding

In [26]:
pd.get_dummies(df_pays_region['Région'])

,Afrique,AmNord,AmSud,Asie,Europe,Oceanie
Zone,,,,,,
Afghanistan,0,0,0,1,0,0
Afrique du Sud,1,0,0,0,0,0
Albanie,0,0,0,0,1,0
Algérie,1,0,0,0,0,0
Allemagne,0,0,0,0,1,0
...,...,...,...,...,...,...
Venezuela (République bolivarienne du),0,0,1,0,0,0
Viet Nam,0,0,0,1,0,0
Yémen,0,0,0,1,0,0


In [27]:
# joindre la matrice 
df_pays_region = df_pays_region.merge(pd.get_dummies(df_pays_region['Région']),how='outer',left_on='Zone',right_on='Zone')
# enlever la colonne région
df_pays_region = df_pays_region.drop(columns=['Région'])

In [28]:
# création d'un deuxième data frame identique ou l'on exclue l'encoding
df_pays = df_pays_region[
    ["Croissance (en millier d'individus)",'Production(en millier de tonnes)',"Dispo intérieure(en millier de tonnes)","Disponibilité alimentaire(en Kg/personne/an)","Balance commeciale (en milier de tonnes)","Population en 2017 (en milier d'individus)","Pib par habitant","% de croissance","Production(en tonne par milier d'habitant)","Balance commerciales(en tonne par milier d'habitant)"]]

### extraire les dataframes en fichier excel

In [29]:
df_pays_region.to_excel('df_pays_region.xlsx')
df_pays.to_excel('df_pays.xlsx')

In [30]:
df_pays[df_pays["Croissance (en millier d'individus)"]>15000]

,Croissance (en millier d'individus),Production(en millier de tonnes),Dispo intérieure(en millier de tonnes),Disponibilité alimentaire(en Kg/personne/an),Balance commeciale (en milier de tonnes),Population en 2017 (en milier d'individus),Pib par habitant,% de croissance,Production(en tonne par milier d'habitant),Balance commerciales(en tonne par milier d'habitant)
Zone,,,,,,,,,,
Inde,16518,3545.0,3661.0,2.22,0.0,1338676.785,1.961338,1.233905,2.648137,0.0


In [31]:
df_pays_region[df_pays_region["Croissance (en millier d'individus)"]>5000]

,Croissance (en millier d'individus),Production(en millier de tonnes),Dispo intérieure(en millier de tonnes),Disponibilité alimentaire(en Kg/personne/an),Balance commeciale (en milier de tonnes),Population en 2017 (en milier d'individus),Pib par habitant,% de croissance,Production(en tonne par milier d'habitant),Balance commerciales(en tonne par milier d'habitant),Afrique,AmNord,AmSud,Asie,Europe,Oceanie
Zone,,,,,,,,,,,,,,,,
"Chine, continentale",7622,18236.0,18161.0,12.33,452.0,1421021.791,8.663126,0.536375,12.833019,0.318081,0,0,0,1,0,0
Inde,16518,3545.0,3661.0,2.22,0.0,1338676.785,1.961338,1.233905,2.648137,0.000000,0,0,0,1,0,0
